In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
import sqlite3 as sq3

import numpy as np
import pandas as pd

import re

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import nltk
from nltk.tokenize import word_tokenize

import matplotlib.pyplot as plt
nltk.download('punkt')


import numpy as np
import pandas as pd

import re

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import nltk
from nltk.tokenize import word_tokenize

import matplotlib.pyplot as plt
# plotting
import seaborn as sns
import matplotlib.pyplot as plt

# nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import nltk
nltk.download('vader_lexicon')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger') 

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [2]:
path = '/content/drive/MyDrive/Colab Notebooks/Sentiment.db'
con = sq3.Connection(path)
query = """
SELECT *
FROM airline;
"""
dataset = pd.read_sql(query, con)
dataset.columns = ['tweet_id','airline_sentiment','airline_sentiment_confidence','negativereason','negativereason_confidence','airline', 'airline_sentiment_gold', 'name', 'negativereason_gold', 'retweetcount', 'text', 'tweet_coord', 'tweet_created', 'tweet_location', 'user_timezone']
dataset = dataset.iloc[1:]
dataset.index = range(len(dataset))
dataset.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweetcount,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0,None,None,Virgin America,None,cairdin,None,0,@VirginAmerica What @dhepburn said.,None,2015-02-24 11:35:52 -0800,None,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,None,0.0,Virgin America,None,jnardino,None,0,@VirginAmerica plus you've added commercials t...,None,2015-02-24 11:15:59 -0800,None,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,None,None,Virgin America,None,yvonnalynn,None,0,@VirginAmerica I didn't today... Must mean I n...,None,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0,Bad Flight,0.7033,Virgin America,None,jnardino,None,0,@VirginAmerica it's really aggressive to blast...,None,2015-02-24 11:15:36 -0800,None,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0,Can't Tell,1.0,Virgin America,None,jnardino,None,0,@VirginAmerica and it's a really big bad thing...,None,2015-02-24 11:14:45 -0800,None,Pacific Time (US & Canada)


In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [4]:
dataset = dataset.drop(columns = ['airline_sentiment_confidence','negativereason','negativereason_confidence','airline', 'airline_sentiment_gold', 'name', 'negativereason_gold', 'retweetcount', 'tweet_coord', 'tweet_created', 'tweet_location', 'user_timezone'])
dataset = dataset.dropna()
dataset.head()

,tweet_id,airline_sentiment,text
0,570306133677760513,neutral,@VirginAmerica What @dhepburn said.
1,570301130888122368,positive,@VirginAmerica plus you've added commercials t...
2,570301083672813571,neutral,@VirginAmerica I didn't today... Must mean I n...
3,570301031407624196,negative,@VirginAmerica it's really aggressive to blast...
4,570300817074462722,negative,@VirginAmerica and it's a really big bad thing...


In [5]:
dataset["airline_sentiment"].value_counts()

negative    9178
neutral     3099
positive    2363
Name: airline_sentiment, dtype: int64

In [6]:
train_df, test_df = train_test_split(dataset, test_size = 0.20)

In [7]:
train_df = train_df.drop(columns = "tweet_id")
test_df = test_df.drop(columns = "tweet_id")

In [8]:
train_df = list(train_df.to_records(index = False))
test_df = list(test_df.to_records(index = False))
train_df[:10]

[('negative', '@USAirways  Instead, the miles are credited to an account that i did not open\n-- My request to merge the accounts is denied'),
 ('neutral', "@SouthwestAir's CEO Kelly draws record crowd to @BWI_Airport Business Partnership breakfast http://t.co/hrvuKtpvn1 http://t.co/MY3dnVBZAZ"),
 ('negative', '@JetBlue How is it possible the only way to contact your fraud department is mailing a physical letter? No telephone? No email? REALLY??'),
 ('negative', "@united it's frustrating, as a frequent flyer, loyal since Continental. Upgrade equals paying 3 times or more on top of purchase ticket."),
 ('positive', '@SouthwestAir thank you so much completely made things right!'),
 ('negative', '@SouthwestAir your "complimentary" hotel vouchers are BS! $79 a night for the crappiest hotel ever! I wouldn\'t wish this on my worst enemy!'),
 ('negative', '@USAirways @USAirwaysCenter My lost bag lands in Hartford at 639p tonight. Will it be delivered to Danbury CT tonight?'),
 ('negative', "@

In [9]:
def remove_links_mentions(tweet):
    link_re_pattern = "https?:\/\/t.co/[\w]+"
    mention_re_pattern = "@\w+"
    tweet = re.sub(link_re_pattern, "", tweet)
    tweet = re.sub(mention_re_pattern, "", tweet)
    return tweet.lower()

In [10]:
train_df = [(label, word_tokenize(remove_links_mentions(tweet))) for label, tweet in train_df]
train_df[:3]

[('negative',
  ['instead',
   ',',
   'the',
   'miles',
   'are',
   'credited',
   'to',
   'an',
   'account',
   'that',
   'i',
   'did',
   'not',
   'open',
   '--',
   'my',
   'request',
   'to',
   'merge',
   'the',
   'accounts',
   'is',
   'denied']),
 ('neutral',
  ["'s",
   'ceo',
   'kelly',
   'draws',
   'record',
   'crowd',
   'to',
   'business',
   'partnership',
   'breakfast']),
 ('negative',
  ['how',
   'is',
   'it',
   'possible',
   'the',
   'only',
   'way',
   'to',
   'contact',
   'your',
   'fraud',
   'department',
   'is',
   'mailing',
   'a',
   'physical',
   'letter',
   '?',
   'no',
   'telephone',
   '?',
   'no',
   'email',
   '?',
   'really',
   '?',
   '?'])]

In [11]:
test_df = [(label, word_tokenize(remove_links_mentions(tweet))) for label, tweet in test_df]
test_df[:3]

[('neutral',
  ['good',
   'to',
   'know',
   '.',
   'perhaps',
   'it',
   'was',
   'execution',
   'then',
   '.']),
 ('neutral',
  ['i', 'absolutely', 'will', '.', 'where', 'can', 'i', 'send', 'it', '?']),
 ('neutral',
  ['its',
   'available',
   'online',
   'i',
   'just',
   'ca',
   "n't",
   'figure',
   'out',
   'how',
   'to',
   'do',
   'it',
   'to',
   'guarantee',
   'same',
   'flight'])]

In [12]:
index2word = ["<PAD>", "<SOS>", "<EOS>"]

for ds in [train_df, test_df]:
    for label, tweet in ds:
        for token in tweet:
            if token not in index2word:
                index2word.append(token)

'an'

In [13]:
word2index = {token: idx for idx, token in enumerate(index2word)}


5

In [14]:
def label_map(label):
    if label == "neutral":
        return 0
    elif label == "negative":
        return 1
    else: 
        return 2

In [15]:
seq_length = 256
def encode_and_pad(tweet, length):
    sos = [word2index["<SOS>"]]
    eos = [word2index["<EOS>"]]
    pad = [word2index["<PAD>"]]

    if len(tweet) < length - 2: 
        n_pads = length - 2 - len(tweet)
        encoded = [word2index[w] for w in tweet]
        return sos + encoded + eos + pad * n_pads 
    else: 
        encoded = [word2index[w] for w in tweet]
        truncated = encoded[:length - 2]
        return sos + truncated + eos

In [16]:
train_encoded = [(encode_and_pad(tweet, seq_length), label_map(label)) for label, tweet in train_df]

In [17]:
test_encoded = [(encode_and_pad(tweet, seq_length), label_map(label)) for label, tweet in test_df]

In [18]:
for i in train_encoded[:3]:
    print(i)

([1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 9, 20, 5, 21, 22, 23, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 1)
([1, 24, 25, 26, 27, 28, 29, 9, 30, 31, 32, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [19]:
batch_size = 32

train_x = np.array([tweet for tweet, label in train_encoded])
train_y = np.array([label for tweet, label in train_encoded])
test_x = np.array([tweet for tweet, label in test_encoded])
test_y = np.array([label for tweet, label in test_encoded])

train_ds = TensorDataset(torch.from_numpy(train_x), torch.from_numpy(train_y))
test_ds = TensorDataset(torch.from_numpy(test_x), torch.from_numpy(test_y))

train_dl = DataLoader(train_ds, shuffle=True, batch_size=batch_size, drop_last=True)
test_dl = DataLoader(test_ds, shuffle=True, batch_size=batch_size, drop_last=True)

In [27]:
class BiLSTM_SentimentAnalysis(torch.nn.Module) :
    def __init__(self, vocab_size, embedding_dim, hidden_dim, dropout) :
        super().__init__()

        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(hidden_dim, 3)

    def forward(self, x, hidden):

        T = 150

        embs = self.embedding(x)
        out, hidden = self.lstm(embs, hidden)
        out = self.dropout(out)
        out = self.fc(out)
        out = F.softmax(out/T, dim = 1)
        out = out[:, -1]
        return out, hidden
    
    def init_hidden(self):
        return (torch.zeros(1, batch_size, 32), torch.zeros(1, batch_size, 32))

In [21]:
model = BiLSTM_SentimentAnalysis(len(word2index), 64, 32, 0.2)
model = model.to(device)

In [22]:
criterion = nn.NLLLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 3e-4)

In [23]:
epochs = 50
losses = []
for e in range(epochs):

    h0, c0 =  model.init_hidden()

    h0 = h0.to(device)
    c0 = c0.to(device)

    for batch_idx, batch in enumerate(train_dl):

        input = batch[0].to(device)
        target = batch[1].to(device)

        optimizer.zero_grad()
        with torch.set_grad_enabled(True):
            out, hidden = model(input, (h0, c0))
            loss = criterion(out, target)
            loss.backward()
            optimizer.step()
    losses.append(loss.item())

In [25]:
batch_acc = []
for batch_idx, batch in enumerate(test_dl):

    input = batch[0].to(device)
    target = batch[1].to(device)

    optimizer.zero_grad()
    with torch.set_grad_enabled(False):
        out, hidden = model(input, (h0, c0))
        _, preds = torch.max(out, 1)
        preds = preds.to("cpu").tolist()
        batch_acc.append(accuracy_score(preds, target.tolist()))

sum(batch_acc)/len(batch_acc)

0.6277472527472527